<a href="https://colab.research.google.com/github/Sapna1624/Car_No.Plate_Detection_Recognition/blob/main/MiniProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr
!pip install imutils   
!pip install "opencv-python-headless<4.3"

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
from google.colab.patches import cv2_imshow

### 1. Read in Image and Grayscale

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/image1.jpg')
print("\n The uploaded image is :- \n")
cv2_imshow(img)
print("\n\n Displaying the uploaded image using matplotlib :- \n")
plt.imshow(img)
plt.show()
print("\n\n Displaying the uploaded image using matplotlib after making the color format as RGB:- \n")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))           #Because matplotlib takes images in BGR format by default.
plt.show()


In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print("\n\n Displaying the grayscale image of the uploaded image using matplotlib :- \n")
plt.imshow(gray)
plt.show()
print("\n\n Displaying the grayscale image of the uploaded image using matplotlib after making the color format as RGB:- \n")
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))
plt.show()


### 2. Apply filter and find edges for localization

In [ ]:
# Performing - Noise reduction using bilateralFilter() 
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) 
# Performing - Edge detection using Canny algorithm 
edged = cv2.Canny(bfilter, 30, 200) 
print("\n\n >>>Displaying the resulted image of the grayscale image using matplotlib after making the color format as RGB:- ")
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

### 3. Find Contours and Apply Mask

In [ ]:
# Finding the contours
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

### Edge Tracking by Hysteresis

In [ ]:
# Finding the location of the contours
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break

In [ ]:
print("The location points of the number plate in the image are :- ")
location

In [ ]:
# Masking the image
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0,255, -1)
# plt.imshow(new_image)
# plt.show()
new_image = cv2.bitwise_and(img, img, mask=mask)
print("\n\n >>>Displaying the masked image of the number plate  using matplotlib :- ")
plt.imshow(new_image)
plt.show()
print("\n\n >>>Displaying the masked image of the number plate  using matplotlib after making the color format as RGB:- ")
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

In [ ]:
#Cropping the image of the number plate from the masked image
(x,y) = np.where(mask==255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
cropped_image = gray[x1:x2+1, y1:y2+1]
print("\n\n >>>Displaying the masked image of the number plate  using matplotlib after making the color format as RGB:- ")
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

### 4. Use Easy OCR To Read Text

In [ ]:
#Specifying the reading language to the EasyOCR as english
reader = easyocr.Reader(['en'])
#Reading the text from the cropped image
result = reader.readtext(cropped_image)
print("\n \n The data obtained from the cropped image of the number plate of the vehicle, uploaded in the image : - \n ", result)
result

### 5. Render Result

In [ ]:
#Taking out the result i.e. only the text on the number plate
text = result[0][-2]
print("\n The text written on the number plate of the vehicle uploaded in the image : - \n", text)

font = cv2.FONT_HERSHEY_SIMPLEX

#Writing the text i.e. the output on the original image
res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)

#Putting a rectangle around the number plate in the original image
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)

print("\n\n >>>Displaying the result of the number plate, in the original image,  using matplotlib after making the color format as RGB:- ")
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
plt.show()